# Example of Prediction

## Data and Library Import

In [1]:
from numpy import array
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import pandas as pd
from numpy import loadtxt
from keras.models import load_model

#load up the trained models (this example only uses amc)
amc_model = load_model('AMCModel.h5')
gme_model = load_model('GMEModel.h5')

## Split Sequence Definiteion to Fit LSTM

In [2]:
#You can probably import the data real time or something?
example_prediction = pd.read_csv(r'./Data/Prediction_example_data.csv', index_col=0, parse_dates=True)

def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

## Model Testing Function

In [3]:
def testing_data_prep(testing_set, model):
    n_steps = 8
    #data here taken from csv columns, have a look at the tetsing set for the format
    price_testing = testing_set['Price'].values
    price_open_testing = testing_set['Open'].values
    price_high_testing = testing_set['High'].values
    price_low_testing = testing_set['Low'].values
    price_vol_testing = testing_set['Vol.'].values
    price_change_testing = testing_set['Change %'].values

    #runs the above split function to make the datset into small arrays with 8 steps each, copy the function for prediction
    #this will need to be done to any prediction, it needs 8 steps behind to predict
    Tprice, y = split_sequence(price_testing, n_steps)
    Tprice = Tprice.reshape((Tprice.shape[0], n_steps, 1))
    
    Topen, y = split_sequence(price_open_testing, n_steps)
    Topen = Topen.reshape((Topen.shape[0], n_steps, 1))
    
    Thigh, y = split_sequence(price_high_testing, n_steps)
    Thigh = Thigh.reshape((Thigh.shape[0], n_steps, 1))
    
    Tlow, y = split_sequence(price_low_testing, n_steps)
    Tlow = Tlow.reshape((Tlow.shape[0], n_steps, 1))
    
    TVol, y = split_sequence(price_low_testing, n_steps)
    TVol = Tlow.reshape((Tlow.shape[0], n_steps, 1))
    
    Tchange, y = split_sequence(price_change_testing, n_steps)
    Tchange = Tchange.reshape((Tchange.shape[0], n_steps, 1))

    #This line takes all of the split arrays and puts them together, shapinng the data structure to fit the lstm
    Example_prediction = np.concatenate((Tprice, Topen, Thigh, Tlow, Tchange), axis = 2)
    #predicts using the correct dataset
    prediction = model.predict(Example_prediction)
    #with the predictions you can model them or if its just one output it
    print(prediction)

In [4]:
testing_data_prep(example_prediction, amc_model)

[[-0.1491504]]
